# DermLIP 피부 질환 진단 시스템

이 노트북은 DermLIP 모델을 사용하여 피부 사진으로부터 질환을 진단하고, 증상과 영향받는 부위, 설명을 제공합니다.

## 모델 정보
- **DermLIP-ViT-B/16**: Vision Transformer 기반 (빠른 속도)
- **DermLIP-PanDerm**: PanDerm 아키텍처 기반 (최고 성능)
- **훈련 데이터**: Derm1M (1,029,761 이미지-텍스트 쌍, 390개 피부 질환)

## 진단 가능한 질환 (12개)
1. 여드름 (Acne)
2. 습진/아토피 (Eczema)
3. 건선 (Psoriasis)
4. 악성 흑색종 (Melanoma)
5. 기저세포암 (Basal Cell Carcinoma)
6. 지루성 각화증 (Seborrheic Keratosis)
7. 주사 (Rosacea)
8. 백반증 (Vitiligo)
9. 헤르페스 (Herpes)
10. 사마귀 (Warts)
11. 접촉성 피부염 (Contact Dermatitis)
12. 피부 진균 감염 (Fungal Infection)

## 1. 라이브러리 설치 및 임포트

In [ ]:
# 필요한 라이브러리 설치 (처음 실행시)
!pip install torch torchvision open_clip_torch pillow matplotlib seaborn

In [ ]:
import torch
import torch.nn.functional as F
import open_clip
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')

# GPU 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'사용 디바이스: {device}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')

## 2. 피부 질환 데이터베이스

In [ ]:
# 피부 질환 데이터베이스 (12개 질환)
SKIN_DISEASE_DATABASE = {
    'acne': {
        'name_ko': '여드름',
        'name_en': 'Acne Vulgaris',
        'affected_areas': ['얼굴', '이마', '뺨', '턱', '등', '가슴'],
        'symptoms': [
            '피부에 작은 붉은 돌기',
            '화농성 병변 (고름이 찬 여드름)',
            '블랙헤드와 화이트헤드 (면포)',
            '피지 과다 분비로 인한 기름진 피부',
            '염증 및 붓기',
            '여드름 자국 또는 흉터'
        ],
        'description': '모낭과 피지선의 만성 염증성 질환입니다. 피지 과다 분비, 모공 막힘, 박테리아 감염이 주요 원인이며, 주로 사춘기에 많이 발생하지만 성인에게도 나타날 수 있습니다.'
    },
    'eczema': {
        'name_ko': '습진 (아토피 피부염)',
        'name_en': 'Atopic Dermatitis / Eczema',
        'affected_areas': ['팔꿈치 안쪽', '무릎 뒤쪽', '손목', '발목', '얼굴', '목', '손'],
        'symptoms': [
            '극심한 가려움증 (특히 밤에 심함)',
            '붉고 염증이 있는 피부',
            '건조하고 비늘처럼 벗겨지는 피부',
            '피부 갈라짐 및 진물',
            '만성적인 긁음으로 인한 피부 두꺼워짐',
            '색소 침착 또는 탈색'
        ],
        'description': '만성 재발성 염증성 피부 질환으로 심한 가려움증이 특징입니다. 유전적 요인, 면역 체계 이상, 피부 장벽 기능 저하 등이 복합적으로 작용합니다.'
    },
    'psoriasis': {
        'name_ko': '건선',
        'name_en': 'Psoriasis',
        'affected_areas': ['팔꿈치', '무릎', '두피', '허리', '손바닥', '발바닥', '손발톱'],
        'symptoms': [
            '은백색 비늘로 덮인 붉은 반점',
            '건조하고 갈라진 피부 (출혈 가능)',
            '가려움증, 작열감 또는 통증',
            '손발톱 두꺼워짐, 변색 또는 함몰',
            '관절 통증 및 뻣뻣함 (건선 관절염)',
            '두피의 비듬 같은 각질'
        ],
        'description': '면역 체계 이상으로 인한 만성 자가면역 질환입니다. 피부 세포가 비정상적으로 빠르게 증식하여 각질이 쌓이고, 스트레스, 감염, 약물 등이 악화 요인이 될 수 있습니다.'
    },
    'melanoma': {
        'name_ko': '악성 흑색종',
        'name_en': 'Malignant Melanoma',
        'affected_areas': ['전신 (특히 등, 다리, 팔, 얼굴)', '손발톱 밑', '점막'],
        'symptoms': [
            '비대칭적인 점 또는 병변 (A: Asymmetry)',
            '불규칙하고 들쭉날쭉한 경계 (B: Border)',
            '색상이 고르지 않거나 여러 색 혼재 (C: Color)',
            '직경 6mm 이상 (D: Diameter)',
            '크기, 모양, 색의 변화 (E: Evolution)',
            '가려움증, 출혈, 궤양'
        ],
        'description': '멜라닌 세포에서 발생하는 악성 종양으로 가장 위험한 피부암입니다. 자외선 노출이 주요 위험 요인이며, 조기 발견 시 완치 가능성이 높지만 전이되면 치명적일 수 있습니다.'
    },
    'basal_cell_carcinoma': {
        'name_ko': '기저세포암',
        'name_en': 'Basal Cell Carcinoma',
        'affected_areas': ['얼굴', '코', '이마', '뺨', '목', '두피', '어깨'],
        'symptoms': [
            '진주 같은 광택이 있는 작은 돌기',
            '중앙이 함몰되고 가장자리가 융기된 병변',
            '잘 낫지 않는 궤양',
            '쉽게 출혈하는 병변',
            '반짝이는 분홍색, 붉은색 또는 흰색 반점',
            '평평하고 흉터 같은 병변'
        ],
        'description': '가장 흔한 피부암으로 느리게 성장하며 다른 장기로 전이는 드뭅니다. 장기간의 자외선 노출이 주요 원인이며, 조기 발견 및 치료 시 완치율이 매우 높습니다.'
    },
    'seborrheic_keratosis': {
        'name_ko': '지루성 각화증',
        'name_en': 'Seborrheic Keratosis',
        'affected_areas': ['얼굴', '가슴', '어깨', '등', '복부'],
        'symptoms': [
            '갈색, 검은색 또는 황갈색의 사마귀 같은 성장물',
            '왁스를 바른 듯한 표면',
            '약간 융기되고 "붙여놓은 듯한" 외관',
            '거칠고 비늘 같은 질감',
            '대부분 통증이나 가려움 없음',
            '크기는 수 mm에서 수 cm까지 다양'
        ],
        'description': '양성 피부 종양으로 나이가 들면서 흔히 발생합니다. 악성이 아니며 건강에 해롭지 않지만, 미용적 이유나 자극을 받는 부위에 있을 경우 제거할 수 있습니다.'
    },
    'rosacea': {
        'name_ko': '주사 (안면홍조)',
        'name_en': 'Rosacea',
        'affected_areas': ['볼', '코', '이마', '턱', '눈 주변'],
        'symptoms': [
            '얼굴 중앙부의 지속적인 홍조',
            '확장된 혈관 (모세혈관 확장)',
            '여드름 같은 붉은 돌기와 농포',
            '피부 비후 (특히 코 - 딸기코)',
            '눈 자극, 충혈, 건조함',
            '화끈거림 또는 따끔거림'
        ],
        'description': '만성 염증성 피부 질환으로 얼굴에 홍조와 혈관 확장을 유발합니다. 정확한 원인은 불명이나 유전, 환경 요인이 관여하며, 스트레스, 알코올, 매운 음식, 온도 변화 등이 악화 요인입니다.'
    },
    'vitiligo': {
        'name_ko': '백반증',
        'name_en': 'Vitiligo',
        'affected_areas': ['얼굴', '손', '팔', '발', '입 주변', '눈 주변', '생식기', '겨드랑이'],
        'symptoms': [
            '피부의 색소 완전 상실',
            '유백색 또는 흰색 반점',
            '대칭적 또는 비대칭적 분포',
            '반점 부위 모발의 조기 백발화',
            '점막(입, 코, 생식기)의 탈색',
            '색소 상실 부위의 점진적 확대'
        ],
        'description': '멜라닌 세포가 파괴되어 피부 색소가 소실되는 자가면역 질환입니다. 건강에는 해롭지 않지만 자외선 차단이 중요하며, 심리적·미용적 영향이 클 수 있습니다.'
    },
    'herpes': {
        'name_ko': '헤르페스',
        'name_en': 'Herpes Simplex',
        'affected_areas': ['입술', '입 주변', '구강 내', '생식기', '엉덩이', '허벅지'],
        'symptoms': [
            '작고 통증이 있는 물집 (수포)',
            '물집이 터진 후 생기는 궤양',
            '발진 전 가려움증, 따끔거림 (전구 증상)',
            '발열, 두통, 근육통',
            '림프절 부종',
            '재발성 발진 (같은 부위)'
        ],
        'description': '헤르페스 바이러스(HSV-1, HSV-2)에 의한 감염입니다. 1형은 주로 구강, 2형은 생식기에 발생하며, 바이러스는 평생 잠복 상태로 남아 있다가 면역력 저하, 스트레스 시 재발합니다.'
    },
    'warts': {
        'name_ko': '사마귀',
        'name_en': 'Verruca / Warts',
        'affected_areas': ['손', '발', '얼굴', '무릎', '팔꿈치', '손발톱 주변'],
        'symptoms': [
            '작고 거칠고 단단한 피부 성장물',
            '표면에 검은 점 (혈전된 모세혈관)',
            '융기된 돌기 또는 평평한 병변',
            '손발바닥 사마귀의 경우 압통',
            '여러 개가 모여 모자이크 패턴 형성',
            '손발톱 변형 (손발톱 주변 사마귀)'
        ],
        'description': '인유두종 바이러스(HPV)에 의한 양성 피부 감염입니다. 전염성이 있으며 직접 접촉이나 간접 접촉으로 전파됩니다. 대부분 양성이며 자연 소실되기도 하지만 치료로 제거 가능합니다.'
    },
    'dermatitis': {
        'name_ko': '접촉성 피부염',
        'name_en': 'Contact Dermatitis',
        'affected_areas': ['손', '얼굴', '목', '팔', '접촉 부위 (국소적)'],
        'symptoms': [
            '붉고 가려운 발진',
            '부종 및 물집',
            '피부 건조, 갈라짐, 벗겨짐',
            '화끈거림 또는 따끔거림',
            '접촉 부위의 명확한 경계',
            '삼출액 및 딱지 형성'
        ],
        'description': '피부가 자극 물질이나 알레르겐에 접촉하여 발생하는 염증 반응입니다. 자극성 접촉 피부염과 알레르기성 접촉 피부염으로 구분되며, 원인 물질 제거가 치료의 핵심입니다.'
    },
    'fungal_infection': {
        'name_ko': '피부 진균 감염 (무좀, 백선)',
        'name_en': 'Tinea / Fungal Infection',
        'affected_areas': ['발', '사타구니', '몸통', '두피', '손톱', '발톱'],
        'symptoms': [
            '원형의 붉은 발진 (가장자리 융기)',
            '중앙은 깨끗하고 가장자리만 활성화',
            '심한 가려움증',
            '피부 벗겨짐, 갈라짐, 각질',
            '손발톱 변색, 두꺼워짐, 부서짐',
            '악취 (특히 발)'
        ],
        'description': '피부사상균에 의한 감염으로 따뜻하고 습한 환경에서 잘 발생합니다. 무좀(발), 완선(사타구니), 체부백선(몸통) 등 발생 부위에 따라 명칭이 다르며, 전염성이 있고 항진균제로 치료합니다.'
    }
}

print(f'등록된 질환 수: {len(SKIN_DISEASE_DATABASE)}개')
print('\n질환 목록:')
for i, (key, info) in enumerate(SKIN_DISEASE_DATABASE.items(), 1):
    print(f'{i:2d}. {info["name_ko"]:20s} ({info["name_en"]})')

## 3. DermLIP 진단 시스템 클래스

In [ ]:
class DermLIPDiagnosisSystem:
    """DermLIP 기반 피부 질환 진단 시스템"""

    def __init__(self, model_name='hf-hub:redlessone/DermLIP_ViT-B-16', device='cuda'):
        self.device = device
        self.model_name = model_name

        print(f'모델 로드 중: {model_name}...')
        
        # 모델 로드
        self.model, _, self.preprocess = open_clip.create_model_and_transforms(
            model_name=model_name,
            device=device
        )
        self.tokenizer = open_clip.get_tokenizer(model_name)
        self.model.eval()

        # 클래스 정보 생성
        self.disease_keys = list(SKIN_DISEASE_DATABASE.keys())
        self.class_descriptions = self._generate_class_descriptions()

        print(f'✓ 모델 로드 완료!')
        print(f'✓ {len(self.disease_keys)}개 피부 질환 진단 가능')

    def _generate_class_descriptions(self):
        """DermLIP 모델용 클래스 설명 생성"""
        descriptions = []
        for key in self.disease_keys:
            info = SKIN_DISEASE_DATABASE[key]
            desc = f"a clinical dermatological photograph of {info['name_en']}"
            descriptions.append(desc)
        return descriptions

    def diagnose(self, image_path, top_k=3):
        """피부 질환 진단 수행"""
        print(f'\n이미지 분석 중: {image_path}')

        # 이미지 로드 및 전처리
        image = Image.open(image_path).convert('RGB')
        image_tensor = self.preprocess(image).unsqueeze(0).to(self.device)

        # 이미지 인코딩
        with torch.no_grad():
            image_features = self.model.encode_image(image_tensor)
            image_features = F.normalize(image_features, dim=-1)

        # 텍스트 인코딩
        text_tokens = self.tokenizer(self.class_descriptions).to(self.device)
        with torch.no_grad():
            text_features = self.model.encode_text(text_tokens)
            text_features = F.normalize(text_features, dim=-1)

        # 유사도 계산
        similarity = (image_features @ text_features.T).squeeze(0)
        probabilities = F.softmax(similarity * 100, dim=0).cpu().numpy()

        # 상위 k개 결과
        top_indices = np.argsort(probabilities)[::-1][:top_k]

        diagnosis_results = []
        for rank, idx in enumerate(top_indices, 1):
            disease_key = self.disease_keys[idx]
            disease_info = SKIN_DISEASE_DATABASE[disease_key]
            confidence = float(probabilities[idx]) * 100

            diagnosis_results.append({
                'rank': rank,
                'confidence': confidence,
                'disease_key': disease_key,
                'disease_info': disease_info
            })

        return diagnosis_results, image

    def display_diagnosis(self, diagnosis_results, image):
        """진단 결과 시각화"""
        # 텍스트 결과 출력
        print('\n' + '='*70)
        print('🏥 진단 결과')
        print('='*70)
        
        for result in diagnosis_results:
            rank = result['rank']
            confidence = result['confidence']
            info = result['disease_info']

            print(f'\n[{rank}위] {info["name_ko"]} ({info["name_en"]})')
            print(f'신뢰도: {confidence:.1f}%')
            print('\n📍 주로 영향받는 부위:')
            for area in info['affected_areas']:
                print(f'   • {area}')
            print('\n🔍 주요 증상:')
            for symptom in info['symptoms']:
                print(f'   • {symptom}')
            print(f'\n📋 설명:')
            print(f'   {info["description"]}')
            
            if rank < len(diagnosis_results):
                print('\n' + '-'*70)
        
        print('\n' + '='*70)
        print('⚠️  주의: 이 결과는 AI 기반 예측이며, 정확한 진단은 전문의와 상담하세요.')
        print('='*70 + '\n')
        
        # 시각화
        fig, axes = plt.subplots(1, 2, figsize=(15, 6))
        
        # 원본 이미지
        axes[0].imshow(image)
        axes[0].axis('off')
        axes[0].set_title('입력 이미지', fontsize=14, fontweight='bold')
        
        # 진단 결과 막대 그래프
        diseases = [f"{r['rank']}. {r['disease_info']['name_ko']}" for r in diagnosis_results]
        confidences = [r['confidence'] for r in diagnosis_results]
        
        colors = plt.cm.viridis(np.linspace(0.3, 0.9, len(diseases)))
        bars = axes[1].barh(range(len(diseases)), confidences, color=colors, alpha=0.8)
        axes[1].set_yticks(range(len(diseases)))
        axes[1].set_yticklabels(diseases)
        axes[1].set_xlabel('신뢰도 (%)', fontsize=12)
        axes[1].set_title('진단 결과', fontsize=14, fontweight='bold')
        axes[1].set_xlim(0, 100)
        axes[1].grid(axis='x', alpha=0.3)
        
        for i, (bar, conf) in enumerate(zip(bars, confidences)):
            axes[1].text(conf + 1.5, i, f'{conf:.1f}%', va='center', fontweight='bold')
        
        plt.tight_layout()
        plt.show()

## 4. 진단 시스템 초기화

In [ ]:
# 진단 시스템 초기화
# 옵션 1: ViT-B/16 모델 (빠름)
diagnosis_system = DermLIPDiagnosisSystem(
    model_name='hf-hub:redlessone/DermLIP_ViT-B-16',
    device=device
)

# 옵션 2: PanDerm 모델 (최고 성능, 더 느림)
# diagnosis_system = DermLIPDiagnosisSystem(
#     model_name='hf-hub:redlessone/DermLIP_PanDerm-base-w-PubMed-256',
#     device=device
# )

## 5. 피부 질환 진단 실행

### 사용 방법:
1. 피부 사진 경로를 `image_path` 변수에 입력
2. 셀 실행
3. 진단 결과 확인 (질환명, 증상, 부위, 설명)

In [ ]:
# 진단할 이미지 경로 설정
image_path = 'path/to/your/skin_image.jpg'  # 여기에 실제 이미지 경로 입력

# 상위 몇 개 결과를 볼지 설정 (기본값: 3)
top_k = 3

# 진단 실행
try:
    diagnosis_results, image = diagnosis_system.diagnose(image_path, top_k=top_k)
    diagnosis_system.display_diagnosis(diagnosis_results, image)
except FileNotFoundError:
    print(f'\n❌ 오류: 이미지 파일을 찾을 수 없습니다: {image_path}')
    print('이미지 경로를 확인하고 다시 시도하세요.\n')
except Exception as e:
    print(f'\n❌ 오류 발생: {e}\n')

## 6. 여러 이미지 일괄 진단 (선택사항)

In [ ]:
# 여러 이미지를 한 번에 진단
image_paths = [
    'path/to/image1.jpg',
    'path/to/image2.jpg',
    'path/to/image3.jpg',
]

for img_path in image_paths:
    try:
        print(f'\n{"="*70}')
        print(f'이미지: {img_path}')
        print('='*70)
        
        diagnosis_results, image = diagnosis_system.diagnose(img_path, top_k=3)
        diagnosis_system.display_diagnosis(diagnosis_results, image)
    except FileNotFoundError:
        print(f'⚠️  이미지를 찾을 수 없음: {img_path}\n')
    except Exception as e:
        print(f'⚠️  오류 발생: {e}\n')

## 7. 사용 가이드

### 진단 절차
1. **이미지 준비**: 진단하고 싶은 피부 부위의 선명한 사진을 준비합니다.
2. **경로 설정**: 5번 셀에서 `image_path` 변수에 이미지 경로를 입력합니다.
3. **진단 실행**: 셀을 실행하면 AI가 이미지를 분석합니다.
4. **결과 확인**: 상위 3개 진단 결과와 각각의 증상, 부위, 설명을 확인합니다.

### 결과 해석
- **신뢰도**: 높을수록 해당 질환일 가능성이 큼
- **주로 영향받는 부위**: 해당 질환이 일반적으로 발생하는 신체 부위
- **주요 증상**: 특징적인 증상들
- **설명**: 질환에 대한 간단한 설명

### 주의사항
⚠️ **이 도구는 참고용이며, 최종 진단은 반드시 피부과 전문의와 상담하세요.**

### 모델 선택
- **ViT-B/16**: 빠른 속도, 일반적인 사용에 적합
- **PanDerm**: 최고 성능, 더 정확한 진단 필요 시 사용

### 팁
- 선명하고 밝은 환경에서 찍은 사진일수록 정확도가 높습니다.
- 병변 부위가 명확하게 보이도록 촬영합니다.
- 여러 각도에서 찍은 사진을 각각 진단해보는 것도 도움이 됩니다.